# Health Study

## Load dataset

I load the dataset and convert the variables to numeric format, preparing for analysis.

In [ ]:
from src.io_utils import load_data, coerce_numeric 

df = coerce_numeric(load_data("data/health_study_dataset.csv"))

#df.info()

## Import libraries

Importing all key libraries for analysis. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from scipy import stats

## Loading custom functions

Importing custom functions and the HealthAnalyzer class from the metrics and viz modules.

In [ ]:
from src.metrics import HealthAnalyzer
from src.metrics import summary_stats, simulated_disease_proportion, ci_mean_normal, bootstrap_mean, linear_regression, pca_analysis
from src import viz as V

analyzer = HealthAnalyzer(df)

## Summary statistics

Calculating basic descriptive stats for health variables, which gives an overview of the dataset's variability.

In [ ]:
# Calculates a summary of statistics for age, weight, height, systolic_bp, cholesterol

stats_df = summary_stats(df)

stats_summary = stats_df.rename(
    columns= {
    "age": "Age (years)",
    "weight": "Weight (kg)",
    "height": "Height (cm)", 
    "systolic_bp": "Systolic Blood Pressure (mmHg)",
    "cholesterol": "Cholesterol (mmol/L)"
})
stats_summary.round(1)


## Histogram of Systolic blood pressure

Histogram is used to show the distribution of systolic blood pressure, helping identify patterns.

In [ ]:
# Histogram of Systolic Blood Pressure

fig, ax = plt.subplots(figsize=(10, 6))
V.hist(ax, df["systolic_bp"], "Distribution of Systolic Blood Pressure", "Systolic Blood Pressure (mmHg)", "Number of Participants", bins=30, edgecolor= "black")
plt.tight_layout()

## Bar plot of smoking status

Bar plot is used to show the proportion of smokers vs non-smokers, which helps understand the group before further analysis.


In [ ]:
# Bar plot of smokers vs non-smokers

total_smokers = df["smoker"].value_counts()

answers = total_smokers.index
counts = total_smokers.values

fig, ax = plt.subplots()

V.bar(ax, answers, counts, "Smoking Status of Participants", "", "Number of Participants", edgecolor="black", width=0.6)
plt.xticks(rotation=0)
plt.tight_layout()

## Boxplot of Weight distribution by gender

Boxplot is used to compare weight distribution between women and men, showing the difference in variability and medians.

In [ ]:
#Boxplot of weight distribution by gender

female_weight = df[df["sex"] == "F"]["weight"]
male_weight = df[df["sex"] == "M"]["weight"]

fig, ax = plt.subplots()

V.boxplot(ax, [female_weight, male_weight], "Weight Distribution by Gender", "Weight (kg)", labels=["Female", "Male"])
plt.tight_layout()

## Scatter plot of the relation between systolic blood pressure and age

Scatter plot is used to see if the older participants tend to have higher blood pressure, including a trend line for general direction.

In [ ]:
# Scatter plot of the relation between systolic blood pressure and age
 
fig, ax = plt.subplots(figsize=(10, 6))

V.scatter(ax, df["age"], df["systolic_bp"], "Relation between Systolic Blood Pressure and Age", "Age (years)", "Systolic Blood Pressure (mmHg)")

# Add a trend line
m, b = np.polyfit(df["age"], df["systolic_bp"], 1)
ax.plot(df["age"], m*df["age"] + b, color="black")

plt.tight_layout()

## Simulated proportion of participants with the disease

The actual proportion of participants with the disease are compared with a simulated random distribution to see whether the observed proportion is higher or lower than expected. 

In [ ]:
# Simulated proportion of participants with the disease

np.random.seed(42)
results = simulated_disease_proportion(df)

print(f"Actual proportion of participants with the disease: {results['disease_count']:.2%}")
print(f"Simulated proportion of participants with the disease: {results['simulated_count']:.2%}")
print(f"The difference: {results['difference']:.2%}")

## True mean of systolic blood pressure 

The true mean of systolic blood pressure is calculated, which is useful for further analysis.

In [ ]:
# Calculate the true mean of systolic blood pressure

true_mean = analyzer.sbp_mean()
print(f"True mean of systolic blood pressure: {true_mean:.2f} mmHg")

## Sample statistics for systolic blood pressure

I randomly sample 40 participants, to calculate the mean, standard deviation, and standard error, which are used in the calculation of the confidence interval.

In [ ]:
# Calculate sample statistics for systolic blood pressure

np.random.seed(35)

sbp = df["systolic_bp"]

n = 40 
x = np.random.choice(sbp, size=n, replace=True)

mean_x = float(np.mean(x))
std = float(np.std(x, ddof=1))
se = std / np.sqrt(n)
mean_x, std, n, se

## Confidence interval for systolic blood pressure

With the help of the sample from the previous cell, a 95% confidence interval is calculated for the true mean systolic bp, showing plausible population values. 

In [ ]:
# Confidesnce interval for systolic blood pressure

lo, hi, mean_x, std, n = ci_mean_normal(x)
(lo, hi), mean_x, std, n, true_mean

## Bootstrap hypothesis test

### Mean difference between smokers vs non-smokers

With the help of the HealthAnalyzer class, the mean difference in systolic blood pressure between smokers and non-smokers is calculated, representing the effect size of the smoking status on blood pressure. 

In [ ]:
# Bootstrap hypothesis test on smokers vs non-smokers

hypothesis = analyzer.smoker_diff()
print(f"Difference: {hypothesis:.2f} mmHg")

### Bootstrap

I test the hypothesis that smokers have higher mean systolic blood pressure than non-smokers, by generating bootstrap samples to compute the confidence interval and p-value.

### Explanation of bootstrap 
- If p < 0.05 there is evidence for a difference
- In this case, the p-value is large
- This means that ther is no significant difference in blood pressure between smokers and non-smokers
- Therefore, the hypothesis that smokers have higher mean blood pressure is not supposed

In [ ]:
# Bootstrap for hypothesis that smokers have higher mean systolic blood pressure than non-smokers

np.random.seed(2024)

smokers = df[df["smoker"] == "Yes"]["systolic_bp"]
nonsmokers = df[df["smoker"] == "No"]["systolic_bp"]

obs_diff, p_boot, (ci_low, ci_high) = bootstrap_mean(smokers, nonsmokers)

print(f"Observed difference: {obs_diff:.2f} mmHg")
print(f"P-value from bootstrap: {p_boot:.3f}")
print(f"95% Confidence Interval: ({ci_low:.2f}, {ci_high:.3f})")

## Linear regression of systolic blood pressure on age and weight

The linear regression is used in order to predict systolic blood pressure, showing r_squared, which is the model's explanatory power and also how these factors influence the prediction. 

In [ ]:
# Linear regression of systolic blood pressure on age and weight

intercept_hat, slope_hat_age, slope_hat_weight, r_squared = linear_regression(df)

print(f"Intercept: {intercept_hat:.2f}")
print(f"Age coefficient: {slope_hat_age:.2f} mmHg/year")
print(f"Weight coefficient: {slope_hat_weight:.2f} mmHg/kg")
print(f"Explained variance: {r_squared:.1%}")

### Residual Analysis 
I perform a residual analysis to check if the model's assumptions are likely, where a random scatter around 0 is the goal because it indicates a good fit and that the model is specified appropriately. 

In [ ]:
# Linear regression residual model 

residuals = df["systolic_bp"] - (intercept_hat + slope_hat_age * df["age"] + slope_hat_weight * df["weight"])
predicted = (intercept_hat + slope_hat_age * df["age"] + slope_hat_weight * df["weight"])

fig, ax = plt.subplots(figsize=(6,4))

V.scatter(ax, predicted, residuals, "Residual plot", "Predicted systolic bp", "Residuals")

ax.axhline(0, color="black", linewidth=1)

## PCA analysis

PCA is applied to simplify the health variables and find the main patterns that explain most of the variation, helping reduce complexity while perserving key info.

In [ ]:
# PCA analysis

cols = ["age", "weight", "height", "systolic_bp", "cholesterol"]

components, variance, pca_model = pca_analysis(df, cols)

print("Variance ratio (%): ", np.round(variance * 100, 2))
pd.DataFrame(components[:5], columns=["PC1", "PC2"]).round(3)